Kaden Vasquez

kadenvasquez2001@gmail.*com*

In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
from google.colab import files

import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import math as math

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Grab Data from Drive

In [ ]:
file_path = '/content/drive/My Drive/Machine Learning Code/Project Work - File/other_final_data.csv'

og_data = pd.read_csv(file_path, quotechar='"')

In [ ]:
(og_data.tail())

,tier,participant_1_champion_name,participant_2_champion_name,participant_3_champion_name,participant_4_champion_name,participant_5_champion_name,participant_6_champion_name,participant_7_champion_name,participant_8_champion_name,participant_9_champion_name,participant_10_champion_name,win
219850,DIAMOND,Seraphine,Teemo,Yone,Aatrox,Xerath,Jax,Viego,Ashe,Xayah,Lillia,0
219851,DIAMOND,Shen,XinZhao,Smolder,Varus,Caitlyn,Darius,Ivern,Yone,Jhin,Neeko,0
219852,DIAMOND,Teemo,Sejuani,Zed,Senna,Janna,Malphite,XinZhao,Gragas,Kalista,Ashe,0
219853,DIAMOND,Riven,Kayn,Smolder,Yone,Azir,Chogath,Nidalee,Talon,Caitlyn,Shaco,0
219854,DIAMOND,Fiora,Velkoz,Akali,Jhin,Rammus,Aatrox,LeeSin,Gragas,Yasuo,Sona,0


In [ ]:
print(np.shape(og_data))

(219855, 12)


In [ ]:
#Add an Index

og_data.insert(0, 'Index', np.arange(0, len(og_data)))

In [ ]:
(og_data.head())

,Index,tier,participant_1_champion_name,participant_2_champion_name,participant_3_champion_name,participant_4_champion_name,participant_5_champion_name,participant_6_champion_name,participant_7_champion_name,participant_8_champion_name,participant_9_champion_name,participant_10_champion_name,win
0,0,IRON,Urgot,Vi,AurelionSol,Ezreal,Chogath,Darius,Brand,Ekko,Varus,FiddleSticks,0
1,1,IRON,Aatrox,Jax,Pantheon,MissFortune,Seraphine,Trundle,Shaco,Swain,Zeri,Lux,1
2,2,IRON,Darius,Briar,Diana,Vayne,Nautilus,Garen,Rammus,Katarina,MissFortune,Morgana,0
3,3,IRON,Teemo,Teemo,Teemo,Teemo,Teemo,MissFortune,MissFortune,MissFortune,MissFortune,MissFortune,0
4,4,IRON,Teemo,Nasus,TwistedFate,Tristana,Nami,Udyr,Viego,Yasuo,Jinx,Blitzcrank,1


In [ ]:
#extract index
data_index = og_data['Index'].values

print(data_index)

[     0      1      2 ... 219852 219853 219854]


In [ ]:
#Seperate out each participant into their champion names:
index = og_data['Index']
tier = og_data['tier']
win = og_data['win']
part_1 = og_data['participant_1_champion_name'].astype(str).values
part_2 = og_data['participant_2_champion_name'].astype(str).values
part_3 = og_data['participant_3_champion_name'].astype(str).values
part_4 = og_data['participant_4_champion_name'].astype(str).values
part_5 = og_data['participant_5_champion_name'].astype(str).values
part_6 = og_data['participant_6_champion_name'].astype(str).values
part_7 = og_data['participant_7_champion_name'].astype(str).values
part_8 = og_data['participant_8_champion_name'].astype(str).values
part_9 = og_data['participant_9_champion_name'].astype(str).values
part_10 = og_data['participant_10_champion_name'].astype(str).values

print(part_10)

['FiddleSticks' 'Lux' 'Morgana' ... 'Ashe' 'Shaco' 'Sona']


##Preparing Data for One Hot Encoding

First, we need to see how Riot is labeling each of these champion names.

One Hot only works if every value in our data set is the same as our total list

In [ ]:
##Search how a character is being labeled by knowing a part of their name.

partial_name = 'Urgot'
filtered_df = og_data[og_data['participant_1_champion_name'].str.contains(partial_name, case=False, na=False)]

print(filtered_df)

         Index     tier participant_1_champion_name  \
0            0     IRON                       Urgot   
116        116     IRON                       Urgot   
340        340     IRON                       Urgot   
466        466     IRON                       Urgot   
576        576     IRON                       Urgot   
...        ...      ...                         ...   
219385  219385  DIAMOND                       Urgot   
219610  219610  DIAMOND                       Urgot   
219648  219648  DIAMOND                       Urgot   
219738  219738  DIAMOND                       Urgot   
219743  219743  DIAMOND                       Urgot   

       participant_2_champion_name participant_3_champion_name  \
0                               Vi                 AurelionSol   
116                         Graves                    Pantheon   
340                           Kayn                  Cassiopeia   
466                       MasterYi                       Sylas   
576      

This will allow us to see if each of our participants has each champion played within the data

This is important to how we approach our One Hot Encoding

In [ ]:
def unique_values_from_csv_column(filename, column_name):
    unique_values = filename[column_name].unique()  # Get unique values from the specified column
    sorted_unique_values = sorted(unique_values)  # Sort unique values alphabetically
    return sorted_unique_values

If the length is 167, then that role uses all of the champions in our data.

In [ ]:
unique_values = unique_values_from_csv_column(og_data, 'participant_1_champion_name')
#print(unique_values)
print(len(unique_values))

167


In [ ]:
unique_values = unique_values_from_csv_column(og_data, 'participant_2_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_3_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_4_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_5_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_6_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_7_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_8_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_9_champion_name')
print(len(unique_values))

unique_values = unique_values_from_csv_column(og_data, 'participant_10_champion_name')
print(len(unique_values))

167
167
167
167
167
167
167
167
167


Great! All lanes use all champions. This makes things immensely easier.

Now that we have our data, lets One Hot Encode this:

## One Hot Encoding

In [ ]:
data_part_1 = pd.DataFrame({
    'category': part_1,
    'value': data_index
})

In [ ]:
def one_hot(data, label):
    # Identify categorical columns
    categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
    # Initialize OneHotEncoder
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    # Fit and transform the data
    encoded_data = encoder.fit_transform(data[categorical_cols])
    # Get the feature names for the one-hot encoded columns
    feature_names = encoder.get_feature_names_out(categorical_cols)
    # Modify feature names with the specified prefix and label
    modified_feature_names = [f"{label}_{name.split('_')[-1]}" for name in feature_names]
    # Convert the encoded data into a DataFrame
    encoded_df = pd.DataFrame(encoded_data, columns=modified_feature_names)
    # Concatenate the original DataFrame with the one-hot encoded DataFrame
    final_data = pd.concat([data.drop(columns=categorical_cols), encoded_df], axis=1)
    return final_data


In [ ]:
one_hot_part_1 = one_hot(data_part_1, 'participant_1')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Let's see if it worked!

In [ ]:
one_hot_part_1

,value,participant_1_Aatrox,participant_1_Ahri,participant_1_Akali,participant_1_Akshan,participant_1_Alistar,participant_1_Amumu,participant_1_Anivia,participant_1_Annie,participant_1_Aphelios,...,participant_1_Yone,participant_1_Yorick,participant_1_Yuumi,participant_1_Zac,participant_1_Zed,participant_1_Zeri,participant_1_Ziggs,participant_1_Zilean,participant_1_Zoe,participant_1_Zyra
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219850,219850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219851,219851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219852,219852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
219853,219853,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Looks good! We know that Urgot is being played in the top lane of this first data match. Let's check to see if he has a "1"

In [ ]:
one_hot_part_1['participant_1_Urgot']

0         1.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
219850    0.0
219851    0.0
219852    0.0
219853    0.0
219854    0.0
Name: participant_1_Urgot, Length: 219855, dtype: float64

Great! Now it's time to run the same process with the rest of the participants.

In [ ]:
#2
data_part_2 = pd.DataFrame({
    'category': part_2,
    'value': data_index
})
one_hot_part_2 = one_hot(data_part_1, 'participant_2')

#3
data_part_3 = pd.DataFrame({
    'category': part_3,
    'value': data_index
})
one_hot_part_3 = one_hot(data_part_1, 'participant_3')

#4
data_part_4 = pd.DataFrame({
    'category': part_4,
    'value': data_index
})
one_hot_part_4 = one_hot(data_part_1, 'participant_4')

#5
data_part_5 = pd.DataFrame({
    'category': part_5,
    'value': data_index
})
one_hot_part_5 = one_hot(data_part_1, 'participant_5')

#6
data_part_6 = pd.DataFrame({
    'category': part_6,
    'value': data_index
})
one_hot_part_6 = one_hot(data_part_1, 'participant_6')

#7
data_part_7 = pd.DataFrame({
    'category': part_7,
    'value': data_index
})
one_hot_part_7 = one_hot(data_part_1, 'participant_7')

#8
data_part_8 = pd.DataFrame({
    'category': part_8,
    'value': data_index
})
one_hot_part_8 = one_hot(data_part_1, 'participant_8')

#9
data_part_9 = pd.DataFrame({
    'category': part_9,
    'value': data_index
})
one_hot_part_9 = one_hot(data_part_1, 'participant_9')

#10
data_part_10 = pd.DataFrame({
    'category': part_10,
    'value': data_index
})
one_hot_part_10 = one_hot(data_part_1, 'participant_10')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_outp

I also need to remove the "value" column from the rest of these.

In [ ]:
def remove_column(df, column_name):
    if column_name in df.columns:
        df.drop(column_name, axis=1, inplace=True)
        return df
    else:
        print(f"Column '{column_name}' not found in the DataFrame.")
        return df

In [ ]:
#1
final_part_1 = remove_column(one_hot_part_1, 'value')

#2
final_part_2 = remove_column(one_hot_part_2, 'value')

#3
final_part_3 = remove_column(one_hot_part_3, 'value')

#4
final_part_4 = remove_column(one_hot_part_4, 'value')

#5
final_part_5 = remove_column(one_hot_part_5, 'value')
#6

final_part_6 = remove_column(one_hot_part_6, 'value')

#7

final_part_7 = remove_column(one_hot_part_7, 'value')

#8

final_part_8 = remove_column(one_hot_part_8, 'value')

#9

final_part_9 = remove_column(one_hot_part_9, 'value')

#10
final_part_10 = remove_column(one_hot_part_10, 'value')

Now let's combine them all into one nice csv.

Let's add back in the other features that we got rid of initially:

In [ ]:
file_path = '/content/drive/My Drive/Machine Learning Code/Project Work - File/final_data_without_champ_data.csv'

other_features = pd.read_csv(file_path, quotechar='"')



In [ ]:
# List every data out
all_data = [index, tier, final_part_1, final_part_2, final_part_3, final_part_4, final_part_5, final_part_6, final_part_7, one_hot_part_8, one_hot_part_9, one_hot_part_10, other_features, win]

# Concatenate the data frames along columns
almost_final_data_one_hot_encoded = pd.concat(all_data, axis=1)

#Rename "value" to "Index"
almost_final_data_one_hot_encoded.rename(columns={'value': 'Index'}, inplace=True)


In [ ]:
almost_final_data_one_hot_encoded

,Index,tier,participant_1_Aatrox,participant_1_Ahri,participant_1_Akali,participant_1_Akshan,participant_1_Alistar,participant_1_Amumu,participant_1_Anivia,participant_1_Annie,...,participant_10_average_win,participant_10_average_damagePerMinute,participant_10_average_effectiveHealAndShielding,participant_10_Tank,participant_10_Fighter,participant_10_Assassin,participant_10_Mage,participant_10_Support,participant_10_Marksman,win
0,0.0,IRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.482653,21.268760,0.436885,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,1.0,IRON,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.522213,26.014024,66.771080,0.0,0.0,0.0,1.0,1.0,0.0,1.0
2,2.0,IRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.500845,23.965494,33.819626,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,3.0,IRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.524843,29.445014,17.680629,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4.0,IRON,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.481137,20.432790,9.412011,1.0,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.528455,38.557803,0.249220,0.0,1.0,0.0,1.0,0.0,0.0,NaN
221294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.542439,38.887912,1.660897,0.0,0.0,0.0,1.0,1.0,0.0,NaN
221295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.510232,35.989909,6.148173,0.0,0.0,0.0,0.0,1.0,1.0,NaN
221296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.508356,30.044121,1.963154,0.0,0.0,1.0,0.0,0.0,0.0,NaN


Time to download this.

In [ ]:
almost_final_data_one_hot_encoded.to_csv('/content/sample_data/my_dataframe.csv', index=False)


files.download('/content/sample_data/my_dataframe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## One Hot Encoding Tutorial:

In [ ]:
# Example DataFrame
data = pd.DataFrame({
    'category': ['A', 'B', 'C', 'A', 'C'],
    'value': [10, 20, 30, 40, 50]
})

# Identify categorical columns
categorical_cols = ['category']

In [ ]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
encoded_data = encoder.fit_transform(data[categorical_cols])

# Convert the encoded data into a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Concatenate the original DataFrame with the one-hot encoded DataFrame
final_data = pd.concat([data.drop(columns=categorical_cols), encoded_df], axis=1)

print(final_data)

   value  category_A  category_B  category_C
0     10         1.0         0.0         0.0
1     20         0.0         1.0         0.0
2     30         0.0         0.0         1.0
3     40         1.0         0.0         0.0
4     50         0.0         0.0         1.0
